# OpenQASM Operations - Code Laboratory

**Section 8: OpenQASM** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### QASM 3.0 (Primary - Modern Syntax)
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `qasm3.dumps()` | `dumps(circuit, **kwargs)` | `str` | Export circuit to QASM 3.0 string |
| `qasm3.loads()` | `loads(qasm_string)` | `QuantumCircuit` | Import circuit from QASM 3.0 string |
| `qasm3.dump()` | `dump(circuit, file)` | `None` | Export circuit to QASM 3.0 file |
| `qasm3.load()` | `load(filename)` | `QuantumCircuit` | Import circuit from QASM 3.0 file |

### QASM 2.0 (Legacy - Wide Compatibility)
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `qasm2.dumps()` | `dumps(circuit)` | `str` | Export circuit to QASM 2.0 string |
| `qasm2.loads()` | `loads(qasm_string)` | `QuantumCircuit` | Import circuit from QASM 2.0 string |
| `qasm2.dump()` | `dump(circuit, file)` | `None` | Export circuit to QASM 2.0 file |
| `qasm2.load()` | `load(filename)` | `QuantumCircuit` | Import circuit from QASM 2.0 file |

---

In [ ]:
"""
Qiskit Code Laboratory - OpenQASM Operations
=============================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit import qasm2  # QASM 2.0 module
from qiskit import qasm3  # QASM 3.0 module
from qiskit.qasm3 import dumps, loads, dump, load, Exporter
import tempfile
import os

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def verify_qasm_roundtrip(qc, version=3, tolerance=1e-10):
    """Verify circuit survives QASM export/import roundtrip."""
    if version == 3:
        qasm_str = qasm3.dumps(qc)
        qc_imported = qasm3.loads(qasm_str)
    else:
        qasm_str = qasm2.dumps(qc)
        qc_imported = qasm2.loads(qasm_str)
    # Compare circuit structure
    return qc.num_qubits == qc_imported.num_qubits and \
           qc.num_clbits == qc_imported.num_clbits

def show_qasm(qc, version=3, label=""):
    """Display QASM output with optional label."""
    qasm_str = qasm3.dumps(qc) if version == 3 else qasm2.dumps(qc)
    print(f"{label}:\n{qasm_str}" if label else qasm_str)
    return qasm_str

def compare_circuits(qc1, qc2, label1="Circuit 1", label2="Circuit 2"):
    """Compare two circuits structurally."""
    print(f"\n{label1}:")
    print(qc1.draw())
    print(f"\n{label2}:")
    print(qc2.draw())
    same = (qc1.num_qubits == qc2.num_qubits and 
            qc1.num_clbits == qc2.num_clbits)
    print(f"\nStructurally equivalent: {same}")
    return same

print("✅ Environment ready")

---

## `qasm3.dumps()`

### Signature
```python
qasm3.dumps(circuit: QuantumCircuit, **kwargs) -> str
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `circuit` | `QuantumCircuit` | Yes | - | The circuit to export |
| `includes` | `list[str]` | No | `["stdgates.inc"]` | Include files for the output |
| `disable_constants` | `bool` | No | `False` | If True, don't use OpenQASM 3 standard constants |

### Returns
`str` - OpenQASM 3.0 formatted string representation of the circuit

### See Also
- README Section: [QASM Export](./README.md#qasm-export)

In [ ]:
# ============================================================
# qasm3.dumps() - BASIC USAGE
# ============================================================

# Simplest case - Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("Original circuit:")
print(qc.draw())

# Export to QASM 3.0 string
qasm_string = qasm3.dumps(qc)
print("\nQASM 3.0 Output:")
print(qasm_string)

# Verify export worked
assert isinstance(qasm_string, str), "Export should return string"
assert "OPENQASM 3.0" in qasm_string, "Should contain QASM 3.0 header"
assert "stdgates.inc" in qasm_string, "Should include stdgates.inc"
print("✅ Basic export verified")

In [ ]:
# ============================================================
# qasm3.dumps() - PARAMETER VARIATIONS
# ============================================================

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Variation 1: Default export
print("[1] Default export:")
print("-" * 40)
qasm_default = qasm3.dumps(qc)
print(qasm_default)

# Variation 2: Custom exporter with no includes
print("\n[2] Export with no includes (custom Exporter):")
print("-" * 40)
exporter = Exporter(includes=[])
qasm_no_includes = exporter.dumps(qc)
print(qasm_no_includes)

# Variation 3: Circuit with parameters
print("\n[3] Circuit with rotation gates:")
print("-" * 40)
qc_rot = QuantumCircuit(1)
qc_rot.rx(np.pi/4, 0)
qc_rot.rz(np.pi/2, 0)
print(qasm3.dumps(qc_rot))

print("✅ Parameter variations demonstrated")

In [ ]:
# ============================================================
# qasm3.dumps() - EDGE CASES & ERRORS
# ============================================================

# Edge Case 1: Empty circuit
print("[1] Empty circuit:")
print("-" * 40)
qc_empty = QuantumCircuit(1)
qasm_empty = qasm3.dumps(qc_empty)
print(qasm_empty)
assert "qubit" in qasm_empty, "Should still declare qubits"

# Edge Case 2: Circuit with only classical bits
print("\n[2] Circuit with classical bits:")
print("-" * 40)
qc_classical = QuantumCircuit(1, 2)
qasm_classical = qasm3.dumps(qc_classical)
print(qasm_classical)

# Edge Case 3: Large circuit
print("\n[3] Multi-qubit circuit (3 qubits):")
print("-" * 40)
qc_large = QuantumCircuit(3, 3)
qc_large.h(0)
qc_large.cx(0, 1)
qc_large.cx(1, 2)
qc_large.measure_all()
print(qasm3.dumps(qc_large))

print("\n✅ Edge cases handled correctly")

In [ ]:
# ============================================================
# qasm3.dumps() - VERIFICATION PATTERN
# ============================================================

def test_qasm_dumps():
    """Test suite for qasm3.dumps()."""
    
    # Test 1: Basic export returns string
    qc = QuantumCircuit(2)
    qc.h(0)
    result = qasm3.dumps(qc)
    assert isinstance(result, str), "Test 1 failed: Should return string"
    
    # Test 2: Contains required headers
    assert "OPENQASM 3.0" in result, "Test 2 failed: Missing QASM header"
    
    # Test 3: Contains qubit declaration
    assert "qubit" in result, "Test 3 failed: Missing qubit declaration"
    
    # Test 4: Gate appears in output
    assert "h" in result.lower(), "Test 4 failed: Gate not in output"
    
    # Test 5: Roundtrip preserves structure
    qc_import = qasm3.loads(result)
    assert qc.num_qubits == qc_import.num_qubits, "Test 5 failed: Qubit count mismatch"
    
    print("✅ All qasm3.dumps() tests passed!")

test_qasm_dumps()

---

## `qasm3.loads()`

### Signature
```python
qasm3.loads(qasm_string: str) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `qasm_string` | `str` | Yes | - | OpenQASM 3.0 formatted string |

### Returns
`QuantumCircuit` - A new circuit created from the QASM string

### Raises
- `QASM3ImporterError`: If the QASM string is malformed

### See Also
- README Section: [QASM Import](./README.md#qasm-import)

In [ ]:
# ============================================================
# qasm3.loads() - BASIC USAGE
# ============================================================

# Simplest case - Import Bell state from QASM 3.0
bell_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
'''

print("QASM 3.0 Input:")
print(bell_qasm)

# Import from QASM string
qc = qasm3.loads(bell_qasm)

print("Imported Circuit:")
print(qc.draw())

# Verify import
assert isinstance(qc, QuantumCircuit), "Should return QuantumCircuit"
assert qc.num_qubits == 2, "Should have 2 qubits"
assert qc.num_clbits == 2, "Should have 2 classical bits"
print("\n✅ Basic import verified")

In [ ]:
# ============================================================
# qasm3.loads() - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Simple circuit (no measurements)
print("[1] Circuit without measurements:")
print("-" * 40)
qasm_simple = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
'''
qc_simple = qasm3.loads(qasm_simple)
print(qc_simple.draw())

# Variation 2: GHZ state (3 qubits)
print("\n[2] GHZ State (3 qubits):")
print("-" * 40)
ghz_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
bit[3] c;
h q[0];
cx q[0], q[1];
cx q[0], q[2];
c[0] = measure q[0];
c[1] = measure q[1];
c[2] = measure q[2];
'''
qc_ghz = qasm3.loads(ghz_qasm)
print(qc_ghz.draw())

# Variation 3: Circuit with rotation gates
print("\n[3] Circuit with rotation gates:")
print("-" * 40)
rot_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[1] q;
rx(pi/4) q[0];
rz(pi/2) q[0];
'''
qc_rot = qasm3.loads(rot_qasm)
print(qc_rot.draw())

print("\n✅ Parameter variations demonstrated")

In [ ]:
# ============================================================
# qasm3.loads() - EDGE CASES & ERRORS
# ============================================================

# Edge Case 1: Minimal valid QASM
print("[1] Minimal valid QASM:")
print("-" * 40)
minimal_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[1] q;
'''
qc_minimal = qasm3.loads(minimal_qasm)
print(f"Qubits: {qc_minimal.num_qubits}, Clbits: {qc_minimal.num_clbits}")

# Error Case 1: Invalid QASM syntax
print("\n⚠️ ERROR DEMONSTRATION:")
print("-" * 40)

# Invalid QASM (missing semicolon)
invalid_qasm = '''OPENQASM 3.0
qubit[1] q;
'''

try:
    qasm3.loads(invalid_qasm)
except Exception as e:
    print(f"Error type: {type(e).__name__}")
    print(f"Error caught correctly for malformed QASM")
    print("✅ Error handling works")

# Error Case 2: Unknown gate
print("\n[2] Unknown gate error:")
print("-" * 40)
try:
    bad_gate_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[1] q;
unknown_gate q[0];
'''
    qasm3.loads(bad_gate_qasm)
except Exception as e:
    print(f"Error for unknown gate: {type(e).__name__}")
    print("✅ Unknown gate error caught")

In [ ]:
# ============================================================
# qasm3.loads() - VERIFICATION PATTERN
# ============================================================

def test_qasm_loads():
    """Test suite for qasm3.loads()."""
    
    # Test 1: Returns QuantumCircuit
    qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
'''
    qc = qasm3.loads(qasm)
    assert isinstance(qc, QuantumCircuit), "Test 1 failed: Should return QuantumCircuit"
    
    # Test 2: Correct qubit count
    assert qc.num_qubits == 2, "Test 2 failed: Should have 2 qubits"
    
    # Test 3: Roundtrip consistency
    qc_orig = QuantumCircuit(2)
    qc_orig.h(0)
    qc_orig.cx(0, 1)
    qasm_str = qasm3.dumps(qc_orig)
    qc_imported = qasm3.loads(qasm_str)
    assert qc_orig.num_qubits == qc_imported.num_qubits, "Test 3 failed: Roundtrip mismatch"
    
    # Test 4: Classical bits preserved
    qasm_with_bits = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
c[0] = measure q[0];
'''
    qc_bits = qasm3.loads(qasm_with_bits)
    assert qc_bits.num_clbits == 2, "Test 4 failed: Classical bits not preserved"
    
    print("✅ All qasm3.loads() tests passed!")

test_qasm_loads()

---

## `qasm3.dump()` and `qasm3.load()` - File Operations

### qasm3.dump() Signature
```python
qasm3.dump(circuit: QuantumCircuit, file: TextIO, **kwargs) -> None
```

### qasm3.load() Signature
```python
qasm3.load(filename: str) -> QuantumCircuit
```

### Parameters
| Function | Parameter | Type | Description |
|----------|-----------|------|-------------|
| `dump` | `circuit` | `QuantumCircuit` | Circuit to export |
| `dump` | `file` | `TextIO` | File object to write to |
| `load` | `filename` | `str` | Path to QASM file |

### See Also
- README Section: [File Operations](./README.md#file-operations)

In [ ]:
# ============================================================
# qasm3.dump() / qasm3.load() - BASIC USAGE
# ============================================================

# Create circuit to save
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("Original circuit:")
print(qc.draw())

# Save to file using dump()
with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
    filename = f.name
    qasm3.dump(qc, f)

print(f"\nSaved to: {filename}")

# Load from file using load()
qc_loaded = qasm3.load(filename)
print("\nLoaded circuit:")
print(qc_loaded.draw())

# Verify roundtrip
assert qc.num_qubits == qc_loaded.num_qubits, "Qubit count mismatch"
assert qc.num_clbits == qc_loaded.num_clbits, "Clbit count mismatch"
print("\n✅ File I/O roundtrip verified")

# Cleanup
os.unlink(filename)

In [ ]:
# ============================================================
# File Operations - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Save and read file contents
print("[1] View saved file contents:")
print("-" * 40)

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
    filename = f.name
    qasm3.dump(qc, f)

# Read file contents to see what was saved
with open(filename, 'r') as f:
    contents = f.read()
print("File contents:")
print(contents)

# Variation 2: Multiple circuits to separate files
print("\n[2] Multiple circuits:")
print("-" * 40)

circuits = [
    ("Bell", QuantumCircuit(2)),
    ("GHZ", QuantumCircuit(3)),
]

# Create the circuits
circuits[0][1].h(0)
circuits[0][1].cx(0, 1)

circuits[1][1].h(0)
circuits[1][1].cx(0, 1)
circuits[1][1].cx(0, 2)

for name, qc in circuits:
    print(f"{name} state: {qc.num_qubits} qubits")

# Cleanup
os.unlink(filename)

print("\n✅ File variations demonstrated")

In [ ]:
# ============================================================
# File Operations - EDGE CASES & ERRORS
# ============================================================

# Error Case 1: File not found
print("[1] FileNotFoundError:")
print("-" * 40)
try:
    qc = qasm3.load('/nonexistent/path/circuit.qasm')
except FileNotFoundError as e:
    print(f"FileNotFoundError caught correctly")
    print("✅ Error handling works")

# Error Case 2: Invalid file content
print("\n[2] Invalid QASM file:")
print("-" * 40)
with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
    f.write("This is not valid QASM!")
    bad_filename = f.name

try:
    qc = qasm3.load(bad_filename)
except Exception as e:
    print(f"Error type: {type(e).__name__}")
    print("✅ Invalid file error caught")

os.unlink(bad_filename)

print("\n✅ Error cases handled correctly")

In [ ]:
# ============================================================
# File Operations - VERIFICATION PATTERN
# ============================================================

def test_file_operations():
    """Test suite for qasm3.dump() and qasm3.load()."""
    
    # Test 1: Roundtrip preserves qubit count
    qc_orig = QuantumCircuit(3, 3)
    qc_orig.h(0)
    qc_orig.cx(0, 1)
    qc_orig.cx(1, 2)
    qc_orig.measure([0, 1, 2], [0, 1, 2])
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
        filename = f.name
        qasm3.dump(qc_orig, f)
    
    qc_loaded = qasm3.load(filename)
    assert qc_orig.num_qubits == qc_loaded.num_qubits, "Test 1 failed: Qubit mismatch"
    
    # Test 2: Classical bits preserved
    assert qc_orig.num_clbits == qc_loaded.num_clbits, "Test 2 failed: Clbit mismatch"
    
    # Test 3: File contains valid QASM header
    with open(filename, 'r') as f:
        content = f.read()
    assert "OPENQASM 3.0" in content, "Test 3 failed: Missing header"
    
    os.unlink(filename)
    print("✅ All file operation tests passed!")

test_file_operations()

---

## `qasm2` Module - QASM 2.0 Operations

### Signature
```python
qasm2.dumps(circuit: QuantumCircuit) -> str
qasm2.loads(qasm_string: str) -> QuantumCircuit
qasm2.dump(circuit: QuantumCircuit, file: TextIO) -> None
qasm2.load(filename: str) -> QuantumCircuit
```

### Key Differences from qasm3
| Feature | QASM 2.0 | QASM 3.0 |
|---------|----------|----------|
| Header | `OPENQASM 2.0;` | `OPENQASM 3.0;` |
| Include | `qelib1.inc` | `stdgates.inc` |
| Qubits | `qreg q[n];` | `qubit[n] q;` |
| Bits | `creg c[n];` | `bit[n] c;` |
| Measure | `measure q -> c;` | `c = measure q;` |

### See Also
- README Section: [QASM 2 vs QASM 3](./README.md#qasm-2-vs-qasm-3)

In [ ]:
# ============================================================
# qasm2 Module - BASIC USAGE
# ============================================================

print("QASM 2.0 Module Operations")
print("=" * 50)

# Create a Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("Original circuit:")
print(qc.draw())

# Export to QASM 2.0 string
qasm2_str = qasm2.dumps(qc)
print("\nQASM 2.0 Output (qasm2.dumps):")
print(qasm2_str)

# Import from QASM 2.0 string
qc_restored = qasm2.loads(qasm2_str)
print("Restored circuit (qasm2.loads):")
print(qc_restored.draw())

# Verify roundtrip
assert qc.num_qubits == qc_restored.num_qubits
assert qc.num_clbits == qc_restored.num_clbits
print("\n✅ QASM 2.0 roundtrip verified")

In [ ]:
# ============================================================
# qasm2 vs qasm3 - SIDE-BY-SIDE COMPARISON
# ============================================================

print("QASM 2.0 vs QASM 3.0 - Side by Side")
print("=" * 60)

# Create test circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Export both versions
qasm2_output = qasm2.dumps(qc)
qasm3_output = qasm3.dumps(qc)

print("[1] QASM 2.0 Output:")
print("-" * 40)
print(qasm2_output)

print("[2] QASM 3.0 Output:")
print("-" * 40)
print(qasm3_output)

# Highlight key differences
print("[3] KEY SYNTAX DIFFERENCES:")
print("-" * 40)
print("""
| Element      | QASM 2.0              | QASM 3.0              |
|--------------|----------------------|----------------------|
| Version      | OPENQASM 2.0;        | OPENQASM 3.0;        |
| Include      | qelib1.inc           | stdgates.inc         |
| Qubit decl   | qreg q[2];           | qubit[2] q;          |
| Bit decl     | creg c[2];           | bit[2] c;            |
| Measurement  | measure q[0] -> c[0] | c[0] = measure q[0]  |
""")

print("💡 QASM 3.0 uses modern assignment-style syntax!")
print("💡 QASM 2.0 uses traditional declarative syntax!")

In [ ]:
# ============================================================
# qasm2 Module - VERIFICATION PATTERN
# ============================================================

def test_qasm2_operations():
    """Verify qasm2 module operations work correctly."""
    
    # Test 1: Basic export/import roundtrip
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    qasm2_str = qasm2.dumps(qc)
    qc_restored = qasm2.loads(qasm2_str)
    assert qc.num_qubits == qc_restored.num_qubits, "Test 1 failed: Qubit mismatch"
    
    # Test 2: QASM 2.0 header present
    assert "OPENQASM 2.0" in qasm2_str, "Test 2 failed: Wrong version"
    
    # Test 3: qelib1.inc present (QASM 2.0 standard library)
    assert "qelib1.inc" in qasm2_str, "Test 3 failed: Wrong include"
    
    # Test 4: qreg/creg declarations (QASM 2.0 style)
    assert "qreg" in qasm2_str, "Test 4 failed: Should use qreg"
    assert "creg" in qasm2_str, "Test 4 failed: Should use creg"
    
    # Test 5: Arrow-style measurement (QASM 2.0)
    assert "->" in qasm2_str, "Test 5 failed: Should use -> for measurement"
    
    # Test 6: File operations
    with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
        filename = f.name
        qasm2.dump(qc, f)
    
    qc_from_file = qasm2.load(filename)
    assert qc.num_qubits == qc_from_file.num_qubits, "Test 6 failed: File roundtrip"
    os.unlink(filename)
    
    print("✅ All qasm2 module tests passed!")

test_qasm2_operations()

---

## ⚠️ TRAP DEMONSTRATION: Static vs Instance Methods

**This is the most common exam trap for OpenQASM!**

See [README for mnemonic](./README.md#static-method-trap)

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Static vs Instance Methods
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: QuantumCircuit.from_qasm_str() vs qc.qasm()")
print("=" * 60)

# Create a test circuit and export
original_qc = QuantumCircuit(2)
original_qc.h(0)
original_qc.cx(0, 1)
qasm_string = qasm3.dumps(original_qc)

print("\nScenario: You have a QASM string and want to create a circuit")
print(f"\nQASM string:\n{qasm_string}")

# ❌ WRONG - Common exam trap!
print("\n❌ WRONG approach (Instance method - FAILS!):")
print("-" * 50)
print("""Code that looks correct but is WRONG:
    qc = QuantumCircuit(2)
    qc.from_qasm_str(qasm_string)  # WRONG! from_qasm_str is STATIC
""")

# Show it actually fails
qc_wrong = QuantumCircuit(2)
has_method = hasattr(qc_wrong, 'from_qasm_str')
print(f"Does instance have from_qasm_str()? {has_method}")
if has_method:
    print("(But calling it as instance method is incorrect usage!)")

# ✅ CORRECT - The right way
print("\n✅ CORRECT approach (qasm3.loads() function):")
print("-" * 50)
print("""Correct code:
    from qiskit import qasm3
    qc = qasm3.loads(qasm_string)  # CORRECT!
""")

# Demonstrate correct approach
correct_qc = qasm3.loads(qasm_string)
print("Result:")
print(correct_qc.draw())

print("\n🔍 THE DIFFERENCE:")
print("- qasm3.dumps(qc) = Export (use existing circuit)")
print("- qasm3.loads(str) = Import (create NEW circuit)")
print("\n💡 See README.md for memory trick!")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: QASM 3.0 Syntax Differences
# ============================================================

print("⚠️ TRAP: QASM 3.0 vs QASM 2.0 Syntax")
print("=" * 60)

print("\n❌ WRONG - QASM 2.0 syntax (will fail in QASM 3.0 loader):")
print("-" * 50)
qasm2_style = '''OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0], q[1];
measure q -> c;
'''
print(qasm2_style)

print("\n✅ CORRECT - QASM 3.0 syntax:")
print("-" * 50)
qasm3_style = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
'''
print(qasm3_style)

# Verify QASM 3.0 works
qc = qasm3.loads(qasm3_style)
print("Imported from QASM 3.0:")
print(qc.draw())

print("\n🔍 KEY DIFFERENCES:")
print("| Feature | QASM 2.0 | QASM 3.0 |")
print("|---------|----------|----------|")
print("| Header | OPENQASM 2.0 | OPENQASM 3.0 |")
print("| Include | qelib1.inc | stdgates.inc |")
print("| Qubits | qreg q[n] | qubit[n] q |")
print("| Bits | creg c[n] | bit[n] c |")
print("| Measure | measure q -> c | c = measure q |")
print("\n💡 Remember: QASM 3.0 = Modern assignment-style syntax!")

---

## COMPARISON: dumps() vs dump() and loads() vs load()

In [ ]:
# ============================================================
# COMPARISON: String vs File Operations
# ============================================================

print("=" * 60)
print("COMPARING: String vs File Operations")
print("=" * 60)

# Create test circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("\nOriginal circuit:")
print(qc.draw())

# Method 1: String operations
print("\n[1] String Operations (dumps/loads):")
print("-" * 40)
print("# Export to string")
print("qasm_str = qasm3.dumps(qc)")
qasm_str = qasm3.dumps(qc)
print(f"Type: {type(qasm_str).__name__}")

print("\n# Import from string")
print("qc_from_str = qasm3.loads(qasm_str)")
qc_from_str = qasm3.loads(qasm_str)
print(f"Type: {type(qc_from_str).__name__}")

# Method 2: File operations
print("\n[2] File Operations (dump/load):")
print("-" * 40)
print("# Export to file")
print("with open('circuit.qasm', 'w') as f:")
print("    qasm3.dump(qc, f)")

with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
    filename = f.name
    qasm3.dump(qc, f)

print(f"File created: {filename}")

print("\n# Import from file")
print("qc_from_file = qasm3.load('circuit.qasm')")
qc_from_file = qasm3.load(filename)
print(f"Type: {type(qc_from_file).__name__}")

# Compare results
print("\n[3] Comparison:")
print("-" * 40)
print(f"From string qubits: {qc_from_str.num_qubits}")
print(f"From file qubits: {qc_from_file.num_qubits}")
print(f"Both equal: {qc_from_str.num_qubits == qc_from_file.num_qubits}")

print("\n💡 Summary:")
print("| Operation | String | File |")
print("|-----------|--------|------|")
print("| Export | dumps() | dump() |")
print("| Import | loads() | load() |")
print("| Use when | In-memory | Persistence |")

os.unlink(filename)

---

## CODE CHALLENGES

In [ ]:
# ============================================================
# CHALLENGE 1: Export and Import Bell State
# ============================================================
# Task: Create a Bell state, export to QASM, import back, verify
# Expected: Imported circuit has 2 qubits, 2 clbits, H and CX gates
# ============================================================

def challenge_1():
    """
    Create a Bell state circuit, export to QASM 3.0 string,
    then import it back.
    
    Returns:
        tuple: (original_qc, qasm_string, imported_qc)
    """
    # YOUR CODE HERE
    # Step 1: Create Bell state circuit with measurements
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    # Step 2: Export to QASM string
    qasm_str = qasm3.dumps(qc)
    
    # Step 3: Import back
    imported = qasm3.loads(qasm_str)
    
    return qc, qasm_str, imported

# Test your solution
original, qasm_string, imported = challenge_1()
print("Original circuit:")
print(original.draw())
print("\nImported circuit:")
print(imported.draw())

# Verification
if (imported.num_qubits == 2 and 
    imported.num_clbits == 2 and 
    "h" in qasm_string.lower() and 
    "cx" in qasm_string.lower()):
    print("\n✅ Challenge 1 PASSED!")
else:
    print("\n❌ Challenge 1 FAILED - try again")

In [ ]:
# ============================================================
# CHALLENGE 2: Parse QASM 3.0 String
# ============================================================
# Task: Import the given QASM string and verify the circuit structure
# Expected: 3 qubit circuit with GHZ state preparation
# ============================================================

ghz_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cx q[0], q[1];
cx q[0], q[2];
'''

def challenge_2(qasm_input):
    """
    Import the QASM string and return the circuit.
    
    Args:
        qasm_input: QASM 3.0 formatted string
    
    Returns:
        QuantumCircuit: The imported circuit
    """
    # YOUR CODE HERE
    qc = qasm3.loads(qasm_input)
    return qc

# Test your solution
qc = challenge_2(ghz_qasm)
print("Imported GHZ circuit:")
print(qc.draw())

# Verification
if qc.num_qubits == 3:
    print("\n✅ Challenge 2 PASSED!")
else:
    print("\n❌ Challenge 2 FAILED - try again")

In [ ]:
# ============================================================
# CHALLENGE 3: File Roundtrip
# ============================================================
# Task: Save a circuit to a file, load it back, verify equality
# Expected: Loaded circuit matches original in structure
# ============================================================

def challenge_3():
    """
    Create a circuit, save to file, load back, return both.
    
    Returns:
        tuple: (original_qc, loaded_qc, are_equal)
    """
    # YOUR CODE HERE
    # Step 1: Create circuit
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    # Step 2: Save to temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
        filename = f.name
        qasm3.dump(qc, f)
    
    # Step 3: Load back
    loaded = qasm3.load(filename)
    
    # Step 4: Check equality
    are_equal = (qc.num_qubits == loaded.num_qubits and 
                 qc.num_clbits == loaded.num_clbits)
    
    # Cleanup
    os.unlink(filename)
    
    return qc, loaded, are_equal

# Test your solution
original, loaded, are_equal = challenge_3()
print("Original:")
print(original.draw())
print("\nLoaded:")
print(loaded.draw())

# Verification
if are_equal:
    print("\n✅ Challenge 3 PASSED!")
else:
    print("\n❌ Challenge 3 FAILED - try again")

---

## OpenQASM 3 Data Types Reference

In [ ]:
# ============================================================
# OpenQASM 3 Data Types (EXAM REFERENCE)
# ============================================================

print("OpenQASM 3 Data Types (EXAM CRITICAL!)")
print("=" * 55)

data_types = [
    ("bit, bit[n]", "Classical bits", "bit[8] reg;"),
    ("int[n]", "Signed integer", "int[32] count = -5;"),
    ("uint[n]", "Unsigned integer", "uint[16] shots = 4096;"),
    ("float[n]", "Floating point (16/32/64)", "float[64] theta = 3.14;"),
    ("angle[n]", "Fixed-point angle [0, 2π)", "angle[32] phi = pi/4;"),
    ("bool", "Boolean true/false", "bool flag = true;"),
    ("duration", "Time duration", "duration d = 100ns;"),
    ("complex[float[n]]", "Complex number", "complex[float[64]] c;"),
]

print("\nData Types:")
print("-" * 55)
for dtype, desc, example in data_types:
    print(f"  {dtype:20} {desc}")
    print(f"      Example: {example}")

print("\nBuilt-in Constants:")
print("-" * 55)
print("  pi    = π ≈ 3.14159...   Usage: rx(pi/2) q[0];")
print("  tau   = 2π ≈ 6.28318...  Usage: rz(tau/4) q[0];")
print("  euler = e ≈ 2.71828...   Mathematical calculations")

print("\nDuration Units:")
print("-" * 55)
print("  ns = nanoseconds")
print("  us = microseconds")
print("  ms = milliseconds")
print("  s  = seconds")
print("  dt = device time unit (backend-specific)")

---

## 📚 Resources

### Conceptual Background
- [README.md](./README.md) - Full explanations, traps, mnemonics

### Quick Reference

| Operation | Method | Example |
|-----------|--------|--------|
| Export to string | `qasm3.dumps()` | `s = qasm3.dumps(qc)` |
| Import from string | `qasm3.loads()` | `qc = qasm3.loads(s)` |
| Export to file | `qasm3.dump()` | `qasm3.dump(qc, f)` |
| Import from file | `qasm3.load()` | `qc = qasm3.load(path)` |

### QASM 3.0 Checklist
- [ ] Header: `OPENQASM 3.0;`
- [ ] Include: `stdgates.inc` (not qelib1.inc!)
- [ ] Registers: `qubit[n]` and `bit[n]` (not qreg/creg!)
- [ ] Measurement: `c = measure q` (assignment style)

---

**Notebook verified with Qiskit 1.x** | Last updated: December 2025